In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
def create_training_samples(image_folder, annotation_folder, target_size):
    training_samples = []

    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotation_folder, image_file.replace(".jpg", ".txt"))
        real_img = cv2.imread(image_path)
        img = cv2.cvtColor(real_img, cv2.COLOR_BGR2RGB)

        with open(annotation_path, 'r') as annotation_file:
            lines = annotation_file.readlines()

            for line in lines:
                data = line.strip().split()
                class_label = int(data[0])
                x_center, y_center, width, height = map(float, data[1:5])


                # Convert YOLO format to (x, y, w, h)
                x = int((x_center - width / 2) * img.shape[1])
                y = int((y_center - height / 2) * img.shape[0])
                w = int(width * img.shape[1])
                h = int(height * img.shape[0])

                # Read the image and crop the region defined by the bounding box

                # cropped_region = img[y:y+h, x:x+w]
                cropped_region = img[max(y, 0):min(y+h, img.shape[0]), max(x, 0):min(x+w, img.shape[1])]

                # Resize the cropped region to the target size
                resized_cropped_region = cv2.resize(cropped_region, target_size)

                # Normalize pixel values to [0, 1]
                normalized_cropped_region = resized_cropped_region / 255.0

                # Append the training sample along with its class label
                training_samples.append((normalized_cropped_region, class_label))

    return training_samples

In [ ]:
# Example usage
image_folder = 'dataset/images'
annotation_folder = 'dataset/labels'

target_size = (224, 224)

training_samples = create_training_samples(image_folder, annotation_folder, target_size)

In [ ]:
# Split the data into training and validation sets
train_samples, val_samples = train_test_split(training_samples, test_size=0.2, random_state=42)

In [ ]:
# Extract features and labels from training samples
X_train, y_train = zip(*train_samples)
X_temp, y_temp = zip(*val_samples)
X_val,X_test,y_val,y_test=train_test_split(X_temp, y_temp,test_size=0.5, random_state=42)

# Convert to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test,y_test= np.array(X_test), np.array(y_test)

In [ ]:
# Create a base VGG19 model
base_vgg_model = VGG19(weights='imagenet', include_top=False)

# Freeze the layers of the InceptionV3 model
base_vgg_model.trainable = False

# Add custom layers for classification
vgg_model = base_vgg_model.output
vgg_model = GlobalAveragePooling2D()(vgg_model)
vgg_model = Dense(1024, activation='relu')(vgg_model)
predictions = Dense(1, activation='sigmoid')(vgg_model)  # Assuming you have 2 classes

# Create the final model
vgg = Model(inputs=base_vgg_model.input, outputs=predictions)


# Compile the model
vgg.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['Accuracy', 'Precision', 'Recall','AUC'])

In [ ]:
# Train the model
vgg.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=16)